In [1]:
import tweepy
import glob
import os
import pandas as pd
import csv
from collections import defaultdict
import datetime

In [2]:
# The API-Key and the API-secret were displayed to you after you registered
API_KEY = 'EhenNZwate84ihy4D3AUtGQB8'
API_SECRET = 'ly0mjcqFj1ra1onG0BCAbDGvMCiERonvVuYXexzBkpjLIBS59W'

# The Access token and the Access secret were displayed when you clicked on "generate"
ACCESS_TOKEN = '378072938-O3yndhLllyWKVfLfXNzloo1PuFIdJapAJQg8sWUY'
ACCESS_SECRET = 'ivrsSXGFYhKfl4w5qNXd9rMUZ7Zj0cIFl9Meqnp5k3e9P'

# Setup tweepy to authenticate with Twitter credentials:
auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
# Create the api to connect to twitter with your credentials
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, compression=True)

In [3]:
# Get tweets that have certain keywords

def get_relevant_tweets(keywords, user_ids, tsv_filename, filters):
    date = datetime.datetime.now()
    
    # Optional: we can define a filter, for example, to ignore retweets
    filter = '-filter:retweets'
    
    # Write them to a file
    tsv_file = tsv_filename + date.strftime("%d-%b-%Y") + '.tsv'

    with open(tsv_file, 'w', encoding='utf-8') as outfile:
        tsv_header = 'Type of Tweet\tCreated at\tUser\tNum_Favorites\tNum_Retweets\tTweet\n'
        outfile.write(tsv_header)    

        for user_id in user_ids:
            print(user_id)
            # Query keywords from specific user
            if filters == 'y':
                query = keywords + ' ' + 'from:' + user_id + ' ' + filter
            else: 
                query = keywords + ' ' + 'from:' + user_id

            # Optional: Determine a date range to limit the results 
            # start_date = '2020-03-14'
            # end_date = '2021-02-19'

            # Optional: Limit the number of tweets  
            # nr_tweets = 100

            # Request all tweets with premium key
            #api.search_full_archive
            
            # Currently it goes only 7 days in advance or something
            tweet_iterator = tweepy.Cursor(api.search, q=query, tweet_mode='extended').items()

            tweets = list(tweet_iterator)

            for i, tweet in enumerate(tweets): 

                # Get tweet of opinion leader
                # attributes: type of tweet, created_at, username, favorite count, retweet count, text.
                text = tweet.full_text.replace('\n', ' ')
                tweet_row = 'Tweet_' + str(i) + '\t' + str(tweet.created_at) + '\t' + str(tweet.user.screen_name) + '\t' + str(tweet.favorite_count) + '\t' + str(tweet.retweet_count) + '\t' + text + '\n'
                outfile.write(tweet_row)    

                # Get replies on that specific tweet using tweet_id
                q_reply = 'to:' + user_id
                reply_tweets = tweepy.Cursor(api.search, q=q_reply, since_id=id, max_id=id, tweet_mode='extended').items()
                for reply in reply_tweets:
                    if hasattr(reply, 'in_reply_to_status_id_str'):
                        if (reply.in_reply_to_status_id_str==tweet.id_str):
                            reply_text = reply.full_text.replace('\n', ' ')
                            reply_row = 'Reply_' + str(i) + '\t' + str(reply.created_at) + '\t' + str(reply.user.screen_name) + '\t' + str(reply.favorite_count) + '\t' + str(reply.retweet_count) + '\t' + reply_text + '\n'                        
                            outfile.write(reply_row)

In [4]:
keywords = '((covid OR corona OR coronavirus) AND (vaccin OR vaccinatie OR vaccins OR vaccinaties OR prik OR vaccineren OR prikken)) OR (coronavaccin OR coronavirusvaccin OR covidvaccin OR coronavaccins OR covidvaccins OR coronavaccinaties OR covidvaccinaties OR coronaprik)'
# keywords_extra = '((covid OR corona) AND (vaccin OR vaccinatie OR vaccins OR vaccinaties OR prik OR vaccineren OR prikken)) OR (coronavaccin OR covidvaccin OR coronavaccins OR covidvaccins OR coronavaccinaties OR covidvaccinaties OR coronaprik OR vaccineren OR vaccinaties OR prik OR prikken OR vaccinatie OR vaccin OR vaccins)'

opinion_leaders = ['P_Bruijning', 'MarcBonten', 'MarcelFVerweij', 'MarionKoopmans',
              'mkeulemans', 'amcvanrossum', 'JochenCals', 'Eefje_de_Bont','IGJnl']
tsv_filename_leaders = 'opinionleaders_tweets_'

get_relevant_tweets(keywords, opinion_leaders, tsv_filename_leaders, 'y')

P_Bruijning
MarcBonten
MarcelFVerweij
MarionKoopmans
mkeulemans
amcvanrossum
JochenCals
Eefje_de_Bont
IGJnl


In [5]:
# ArtsenC following
keywords = '((covid OR corona OR coronavirus) AND (vaccin OR vaccinatie OR vaccins OR vaccinaties OR prik OR vaccineren OR prikken)) OR (coronavaccin OR coronavirusvaccin OR covidvaccin OR coronavaccins OR covidvaccins OR coronavaccinaties OR covidvaccinaties OR coronaprik)'
# keywords_extra = '((covid OR corona) AND (vaccin OR vaccinatie OR vaccins OR vaccinaties OR prik OR vaccineren OR prikken)) OR (coronavaccin OR covidvaccin OR coronavaccins OR covidvaccins OR coronavaccinaties OR covidvaccinaties OR coronaprik OR vaccineren OR vaccinaties OR prik OR prikken OR vaccinatie OR vaccin OR vaccins)'

df = pd.read_csv('list of interesting tweeters COVID.csv', sep=';')
df = df[df['Important'].notna()]
interesting_tweeters = df['Following'].tolist()

tsv_interesting_tweeters = 'artsenC_following_covidvaccin_tweets_'
get_relevant_tweets(keywords, interesting_tweeters, tsv_interesting_tweeters, 'y')

Herstel_NL
CoronaOnderzoek
OudersDe
BuuropReis
BertHuisjes
WNLOpiniemakers
WNLVandaag


Rate limit reached. Sleeping for: 837


ArmandGirbes
ClaimedIn
_GezondVerstand
irene_kamp
aukew
JKNL
tussenmeer
leonardwitkamp
DaanDeWit
Ronald_Roothans
huisartsleiden
hapelensrob
ZelfzorgCovid19
MdeGruyter
Sabrina_Nachter
Gezondheidszorg
DStuijver
medischcontact
BartVanHelden1
hetAVL
Infocadl2015
VoedingLeeft
GvRookvrij
RutgervdNoort
arts_leefstijl
J_van_Roosmalen
BorgerPieter


Rate limit reached. Sleeping for: 837


hugovdwedden
artsJannes
Wdekanter
corenona20
Alicia1984N


Rate limit reached. Sleeping for: 838


dutchanddonts


Rate limit reached. Sleeping for: 837


MauricevdBosch
BertMesselink
BetermetBUi
nvzziekenhuizen
KNMP
KNMG
Peter_Voshol
m2_nl
huisartslieneke
GioMeijer
Fitdieet
Coach_Mindtrain
RF_HFC
koosdirkse
DokterJanneke
EsthervanFenema


In [6]:
df = pd.read_csv('ikvaccineer_medical.tsv', delimiter='\t')
ikvaccineer_medicals = df['screen_name'].tolist()
tsv_interesting_tweeters = 'medicals_covidvaccin_tweets_retweets_'
get_relevant_tweets(keywords, ikvaccineer_medicals, tsv_interesting_tweeters, 'n')

daargaanweweetj
Bartissimus


Rate limit reached. Sleeping for: 836


Nimnyana
huisdoktertim
dwingel58
doktertweets
maasvdn
ForensischArts


In [7]:
# # Try filter ppl who have health-related job in their bio: “arts”, “verpleegkundige” + #ikvaccineer or #ikvaccineerniet
# # Download all tweets with  (covid and vaccin) + comments on those
# # NO RETWEETS

# Get ppl with #ikvaccineer or #ikvaccineerniet
def ik_vaccineer_or_not_people(q, filename, num):
    language = 'nl'
    query = q
    users = list(tweepy.Cursor(api.search_users, q=query, iso_language_code=language, tweet_mode='extended', lang = 'nl').items(num))
    with open(filename, 'a', encoding='utf-8') as outfile:
#         tsv_header = 'screen_name\tname\tbio\n'
#         outfile.write(tsv_header)
        
#     with open(filename, 'a', encoding='utf-8') as outfile:
        for user in users:
            print(user.screen_name)
            row = user.screen_name + '\t' + user.name + '\t' + user.description + '\n'
            outfile.write(row)
            
# ik_vaccineer_or_not_people('#ikvaccineer', 'ikvaccineer_people.tsv', 1000)
# ik_vaccineer_or_not_people('#ikvaccineerniet', 'ikvaccineerniet_people.tsv', 100)

In [8]:
def ik_vaccineer_or_not_medical(ikvaccfile, keywords, filename):
    ikvaccdf = pd.read_csv(ikvaccfile, delimiter='\t')
    ikvacc_med = ikvaccdf[ikvaccdf['name'].str.contains(keywords) | ikvaccdf['bio'].str.contains(keywords)]
    ikvacc_med = ikvacc_med.drop_duplicates()
    
    with open(filename, 'a', encoding='utf-8') as outfile:
        # tsv_header = 'screen_name\tname\tbio\n'
        # outfile.write(tsv_header)
        for index, row in ikvacc_med.iterrows():    
            rows = row[0] + '\t' + row[1] + '\t' + row[2] + '\n'
            outfile.write(rows)

In [9]:
# keywords = "arts|dokter|verpleeg|medisch|immunoloog|vaccinoloog|epidemioloog|geneeskundige|geriaters"

# ik_vaccineer_or_not_medical('ikvaccineer_people.tsv', keywords, 'ikvaccineer_medical.tsv')
# ik_vaccineer_or_not_medical('ikvaccineerniet_people.tsv', keywords, 'ikvaccineerniet_medical.tsv')

In [10]:
# Merge files
# def merge_files(merge_file_list, outname):
#     content = []
#     for file in merge_file_list:
#         with open(file, "r", encoding='utf-8') as f:
#             lines = f.read().splitlines()
#             if len(lines) > 1:
#                 content+=lines

#     df = pd.DataFrame([sub.split('\t') for sub in content])
#     df.columns = df.iloc[0]
#     df = df[1:]
#     df.to_csv(outname, sep="\t")

In [11]:
def get_follow_file(acc_name, tsv_filename):
    date = datetime.datetime.now()
    tsv_file = tsv_filename +  date.strftime("%d-%b-%Y") + '.csv'
    with open(tsv_file, 'w', encoding='utf-8') as outfile:
        tsv_header = 'Following\n'
        outfile.write(tsv_header)

        for following in tweepy.Cursor(api.friends, screen_name=acc_name).items():
            outfile.write(following.screen_name +'\n')    

In [12]:
# get_follow_file('ArtsenC', 'ArtsenC_following_')
# get_follow_file('RIVM', 'RIVM_following_')

In [13]:
# # getting the retweeters 
# retweets_list = api.retweets(ID) 
  
# # printing the screen names of the retweeters 
# for retweet in retweets_list: 
#     print(retweet.user.screen_name) 